# Stock Data Collection & Preparation

This notebook finds data on all the US listed companies, and sorts through a list of stocks which can be found on **alpha vantage**. The raw list is cleaned to remove ETFs, remove companies which cannot be found on Yahoo Finance, and save the final data as a csv file to be read from in other documents.

In [1]:
import pandas as pd
import yfinance as yf
import requests
from IPython.display import display, clear_output
import time

In [2]:
# Defining some preliminary functions

def save_dataframe_to_csv(df, filename):
    '''Function to save stock data to csv (to prevent exceeding rate limit)'''
    try:
        df.to_csv(filename, index=False)  # Set index=False if you do not want to save the index as a separate column
        print(f"DataFrame is successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving DataFrame to CSV: {e}")


def get_stock_sector(symbol):
    '''Fetches sector information for a given stock symbol using yfinance.'''
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        sector = info.get('sector', 'Sector information not available')
        if sector != 'Sector information not available':
            return sector, True  # Return True if sector was fetched successfully
        else:
            return sector, False
    except Exception as e:
        print(f"Failed to fetch sector for {symbol}: {e}")
        return 'Sector information not available', False

In [3]:
# Accessing and saving US Stock data 

API_KEY = 'IU8VJCO75R85LMTH'
URL = "https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=" + API_KEY

# Make the API call
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    # Assuming the response is a CSV, convert it into a DataFrame
    from io import StringIO
    data = StringIO(response.text)
    df = pd.read_csv(data)
    
    # Filter for active US stocks
    us_stocks_df = df[df['exchange'].isin(['NYSE', 'NASDAQ', 'AMEX']) & (df['status'] == 'Active')]
    save_dataframe_to_csv(us_stocks_df, 'us_equities.csv')
else:
    print("Failed to fetch data: ", response.status_code)

us_stocks_df

# Assuming the dataframe of US stocks is already saved as a csv
df = pd.read_csv('us_equities.csv')
df

DataFrame is successfully saved to us_equities.csv


,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
3,AACI,Armada Acquisition Corp I,NASDAQ,Stock,2021-11-10,NaN,Active
4,AACIU,Armada Acquisition Corp I - Units (1 Ord & 1/2...,NASDAQ,Stock,2021-08-13,NaN,Active
...,...,...,...,...,...,...,...
8355,ZXYZ-A,NASDAQ SYMBOLOGY TEST,NASDAQ,Stock,2016-01-19,NaN,Active
8356,ZXZZT,NASDAQ TEST STOCK,NASDAQ,Stock,2006-07-10,NaN,Active
8357,ZYME,Zymeworks BC Inc,NASDAQ,Stock,2017-04-28,NaN,Active
8358,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active


In [4]:
# Using only Stocks (not ETFs)

us_stocks_df = df[df['assetType'] == 'Stock'].reset_index(drop=True)

In [5]:
# Cleaning data by removing NaNs, dropping duplicates and anything other than Class A

df_cleaned = us_stocks_df.dropna(subset=['name'])
df_unique = df_cleaned.drop_duplicates(subset='name', keep='first')
df_filtered = df_unique[
    (df_unique['name'].str.endswith(' - Class A')) |  # Keep if it ends with ' - Class A'
    (~df_unique['name'].str.contains(' - '))           # Keep if there's no '-' indicating a subclass or unit
].reset_index(drop=True)

df_filtered

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
3,AACI,Armada Acquisition Corp I,NASDAQ,Stock,2021-11-10,NaN,Active
4,AACT,Ares Acquisition Corporation II - Class A,NYSE,Stock,2023-06-12,NaN,Active
...,...,...,...,...,...,...,...
5789,ZWS,Zurn Elkay Water Solutions Corp,NYSE,Stock,2012-03-29,NaN,Active
5790,ZXYZ-A,NASDAQ SYMBOLOGY TEST,NASDAQ,Stock,2016-01-19,NaN,Active
5791,ZYME,Zymeworks BC Inc,NASDAQ,Stock,2017-04-28,NaN,Active
5792,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active


In [6]:
''' Adding a 'sector' column to the data '''
# Initialize the column with default values (or empty strings if preferred)
total = len(df_filtered['symbol'])
count = 0
fail_count = 0
start_time = time.time()

for index, row in df_filtered.iterrows():
    sector, success = get_stock_sector(row['symbol'])
    df_filtered.at[index, 'sector'] = sector  # Update the 'sector' column directly
    count += 1

    if not success:
        fail_count += 1

    elapsed_time = time.time() - start_time
    remaining_time = ((elapsed_time / count) * (total - count))
    remaining_minutes = int(remaining_time // 60)
    remaining_seconds = int(remaining_time % 60)

    clear_output(wait=True)
    display(f"Processed {count}/{total} tickers. ({(count/total) * 100:.2f}%) - "
            f"Estimated Time Remaining: {remaining_minutes} minutes {remaining_seconds} seconds. "
            f"Failures: {fail_count}")

print("Processing complete. Total Failures:", fail_count)

'Processed 5794/5794 tickers. (100.00%) - Estimated Time Remaining: 0 minutes 0 seconds. Failures: 476'

Processing complete. Total Failures: 476


In [7]:
# Removing stocks with no sector information (as there probably is little other information on them)
df_filtered = df_filtered[df_filtered['sector'] != 'Sector information not available']
df_filtered

save_dataframe_to_csv(df_filtered, 'us_stock_data.csv')

DataFrame is successfully saved to us_stock_data.csv
